## Modelo de Clasificación sin pasar por KG

In [1]:
import numpy as np
import pandas as pd
from ampligraph.utils import restore_model
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

df = pd.read_pickle("../Data/reducedDataset001.pkl")
df["train"] = df.Sujeto > "S14"
model = restore_model("../Data/KGEmbedModel004.pkl")

df.Sujeto = df.Sujeto.values.astype(str)
df.Sujeto = df.Sujeto.str.replace("S","")
df.Sujeto = df.Sujeto.values.astype(np.int64)

/home/jaime/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jaime/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jaime/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jaime/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

In [2]:
x_train = df[df["train"]][["ECG", "EMG", "EDA", "TEMP", "RESP"]].values
y_train = df[df["train"]]["label"].values
x_test = df[~df["train"]][["ECG", "EMG", "EDA", "TEMP", "RESP"]].values
y_test = df[~df["train"]]["label"].values

In [9]:
clf_model = RandomForestClassifier(max_depth=7, random_state=0, n_jobs=-1, n_estimators=2000)
clf_model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [10]:
prediction = clf_model.predict(x_test)
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           1       0.56      0.68      0.61      3246
           2       0.42      0.31      0.36      1877
           3       0.00      0.00      0.00      1004
           4       0.36      0.46      0.40      2250

    accuracy                           0.46      8377
   macro avg       0.33      0.36      0.34      8377
weighted avg       0.41      0.46      0.42      8377

[[2193  108  109  836]
 [ 603  580    0  694]
 [ 445  189    0  370]
 [ 701  503    0 1046]]


In [11]:
joblib.dump(clf_model, '../Data/RFClassifier.pkl')

['../Data/RFClassifier.pkl']